In [1]:
import sqlite3
con1 = sqlite3.connect('DataPrisoners.db')
cur1 = con1.cursor()
print('DELIVERERS')
for row in cur1.execute('''Select * from Deliverer, User where user.ssn = deliverer.ssn
                            '''):
    print(row)
print('------')
print('CUSTOMERS')
for row1 in cur1.execute('''Select * from Customer, User where user.ssn = customer.ssn
'''):
    print(row1)     
print('------')
print('STOCKERS')    
for row2 in cur1.execute('select * from Stocker, User where user.ssn = stocker.ssn'):
    print(row2)
    

DELIVERERS
(458769000, 458769000, 'Mora', 'Gasseo', 'notnull1')
(534789099, 534789099, 'Hafsah', 'Shah', 'perfect44')
(553349023, 553349023, 'Kulsum', 'Shahid', '00000000')
(786345123, 786345123, 'Karem', 'Furkan', 'asdfghj')
(786454122, 786454122, 'Hamji', 'Wana', 'njk9mko0')
------
CUSTOMERS
(134234098, 'Lalu Khet, 54', 5523490000, 134234098, 'Bic', 'Saikal', 'abcdefgh')
(190001234, 'Ghani Sons, 99', 5512021111, 190001234, 'Manuni', 'Sansa', 'abc123abc')
(222123456, 'Bandar Gah, 98', 5212347699, 222123456, 'Gabbar ', 'Jhansi', 'efg234gfh')
(278560345, 'Sind Bad, 23', 5098765432, 278560345, 'Tag', 'Bahr', 'bbc555bbc')
(349657801, 'Mus Kar, 67', 5556784321, 349657801, 'Shipta', 'Falan', 'wwyxtt')
(384762384, 'Fin Pall, 12', 5768635674, 384762384, 'Hyein', 'Kim', 'password')
------
STOCKERS
(890756145, 890756145, 'Pablo', 'Sanchez', '1q2w3e4r5t')
(900089723, 900089723, 'Tata', 'Mike', 'zxc1234v')
(906745342, 906745342, 'Hashim', 'Khan', '13579cv')
(937586239, 937586239, 'Deniz', 'Topay'

In [2]:
import random
import sqlite3
import PySimpleGUI as sg
from datetime import datetime
sg.theme('BluePurple')
# connect to the DB
con = sqlite3.connect('DataPrisoners.db')
cur = con.cursor()

# global variables
login_user_ssn = -1
login_user_name = -1
login_user_type = -1
#--------------------------------------------------------------------------
#window
def window_login():
    
    layout = [[sg.Text('Welcome to Hilbert Plush Store. Please enter your information.')],
              [sg.Text('SSN:',size=(10,1)), sg.Input(size=(10,1), key='ssn')],
              [sg.Text('Password:',size=(10,1)), sg.Input(size=(10,1), key='password')],
              [sg.Button('Login')]]

    return sg.Window('Login Window', layout)
#--------------------------------------------------------------------------
def window_customer():
    
    layout = [[sg.Text('Welcome ' + login_user_name)],
              [sg.Button('Shop')],
              [sg.Button('Cart')],
              [sg.Button('Orders')],
              [sg.Button('Logout')]]

    return sg.Window('Customer Window', layout)
#--------------------------------------------------------------------------
def window_deliverer():
    
    layout = [[sg.Text('Welcome ' + login_user_name)],
              [sg.Button('Paid Carts')],
              [sg.Button('Ongoing Deliveries')],
              [sg.Button('Completed Deliveries')],
              [sg.Button('Logout')]]

    return sg.Window('Deliverer Window', layout)
#--------------------------------------------------------------------------
def window_stocker():
    
    layout = [[sg.Text('Welcome ' + login_user_name)],
              [sg.Button('Your One-Time Jobs')],
              [sg.Button('Update Product Information')],
              [sg.Button('Insert New Product')],
              [sg.Button('Logout')]]

    return sg.Window('Stocker Window', layout)
#---------------------------------------------------
def window_shop():
    products = []
    brands = []
    for row in cur.execute('''SELECT Bname
                              FROM Brand'''):
        brands.append(row)
    for row in cur.execute('''SELECT Pnumber, Bname, Description, Pname, Price
                              FROM Product'''):
        products.append(row)
    layout = [[sg.Text('Brands:'),sg.Combo(brands,size=(25,7),key='Bname'),sg.Button('List Products')],
              [sg.Text('Choose a plushie:')],
              [sg.Listbox((), size=(40,10), key='product')],
              [sg.Text('Amount:'),sg.Input(size=(3,1),key='amount'),sg.Button('Add to Cart')],
              [sg.Button('Back')]]

    return sg.Window('Shopping Window', layout)

#-----------------------------------------------------------------------------------------
def window_deliveries():
    global login_user_ssn
    del_carts = []
    for row in cur.execute('''select customer_id, cart_id from cart 
                                where deliverer_id = ? 
                                and delivery_status = 'on delivery' ''',(login_user_ssn,)):
        del_carts.append(row)
    layout = [[sg.Text('Your Carts on Delivery')],
              [sg.Combo(del_carts,size = (25,12),key = 'dcarts'),sg.Button('Show Cart')],
              [sg.Text('CustomerID CartID Address')],
               [sg.Listbox((),size = (35,20),key = 'addcarts'), sg.Button('Mark as Completed')],
                [sg.Button('Main Page')]]
    
    return sg.Window('Deliveries Window', layout)
       
#-----------------------------------------------------------------------------------------
def window_completed_deliveries():
    global login_user_ssn
    completed = []
    for row in cur.execute('''select C.customer_id, C.Cart_id, R.Address, T.Plate_no
                            from Cart C, Customer R, Truck_Used T
                            where C.Customer_id = R.ssn and C.customer_id = T.customer_id
                            and C.cart_id = T.Cart_id and C.delivery_status = 'completed' and 
                            c.deliverer_id = ?''',
                            (login_user_ssn,)):
        completed.append(row)
    
    layout = [[sg.Text('Deliveries you have completed:')],
              [sg.Text('CustomerID Cart    Address      Plate_no')],
              [sg.Listbox(completed, size =(35,20), key = 'completed')],
              [sg.Button('Main Page')]]

    return sg.Window('Completed Deliveries', layout)
#-----------------------------------------------------------------------------------------
def button_show_cart(values): 
    global login_user_ssn
    d_cart= values['dcarts']
    print(values)
    if d_cart == '':
        sg.popup('Please choose a cart first!')
    else:
        customer_id = d_cart[0]
        cart_id = d_cart[1]
        add_carts = []
        for row in cur.execute('''SELECT C.customer_id, C.cart_id, R.Address
                                  FROM Cart C, Customer R
                                  WHERE R.ssn = c.customer_id and C.deliverer_id =?
                                  and C.customer_id = ? and C.cart_id = ?
                                  and delivery_status = 'on delivery' ''',(login_user_ssn,customer_id,cart_id)):
            add_carts.append(row)
        
        window.Element('addcarts').Update(values=add_carts)
        
#-----------------------------------------------------------------------------------------
def button_markascompleted(values):
    delivery = values['addcarts']
    if delivery == []:
        sg.popup('Choose a cart first!') 
    else:
        customer_id = delivery[0][0]
        cart_id = delivery[0][1]
        cur.execute('''update cart set delivery_status = 'completed'
                        where customer_id =? and cart_id = ?
                        ''', (customer_id,cart_id))
        con.commit()
        
        del_carts =[]
        for row in cur.execute('''select customer_id, cart_id from cart 
                                where deliverer_id = ? 
                                and delivery_status = 'on delivery' ''',(login_user_ssn,)):
            del_carts.append(row)
        add_carts = []
#        for row in cur.execute('''SELECT C.customer_id, C.cart_id, R.Address
  #                                FROM Cart C, Customer R
      #                            WHERE R.ssn = c.customer_id and C.deliverer_id =?
      #                            and C.customer_id = ? and C.cart_id = ?
       #                           and delivery_status = 'on delivery' ''',(customer_id,cart_id,login_user_ssn)):
         #   add_carts.append(row)
        
        window.Element('dcarts').Update(values = del_carts)
        window.Element('addcarts').Update(values = add_carts)
        sg.popup('Delivery successfully completed!')
   
#-----------------------------------------------------------------------------------------

def button_list_products(values): 
    brands= values['Bname']
    if brands == '':
        sg.popup('Please choose a brand.')
    else:
        bname = brands[0]
        products = []
        for row in cur.execute('''SELECT Pnumber, P.Bname, P.Description, Pname, Price
                                  FROM Brand B, Product P
                                  WHERE P.Bname = B.Bname
                                  AND B.Bname = ?''', (bname,)):
            products.append(row)
        
        window.Element('product').Update(values=products)

#-----------------------------------------------------------------------------------------
def window_cart():
    global cart_id
    global login_user_ssn

    cart = []
    t_price = 0
    n_product = 0
    count = 0
    for row in cur.execute('''Select P.Pnumber,P.Bname, P.Pname, P.price, I.amount, C.Total_price, C.Number_of_Products
                              From Product P, Product_In_Cart I, Cart C
                              Where P.Pnumber = I.Pnumber and C.Cart_ID = I.Cart_ID
                                    and C.Customer_ID = I.Customer_ID
                                    and C.Cart_ID = ? and C.customer_id = ?
                                    ''', (cart_id,login_user_ssn)):
        cart.append((row[0],row[1],row[2],row[3],row[4]))
        count += 1
        t_price += row[5]
        n_product += row[6]
    if count!= 0:    
        t_price = t_price/count
        n_product = n_product/count
    layout = [[sg.Text('Cart ID:'),sg.Text(cart_id)],
              [sg.Listbox(cart,size=(40,12),key='cart'),sg.Button('Remove a Product'), sg.Button('Complete Order')],
              [sg.Text('No. of Products:'),sg.Text(str(n_product),key='nproduct')], 
              [sg.Text('Total:'),sg.Text('$'+str(t_price),key='tprice')],
              [sg.Button('Back')]]
    
    return sg.Window('My Cart', layout)
                
#-----------------------------------------------------------------------------------------
def window_orders():
    global login_user_ssn
    orders = []
    for row in cur.execute('''select date, cart_id, number_of_products, total_price, delivery_status
                            from Cart 
                            where customer_id = ?''',
                            (login_user_ssn,)):
        orders.append(row)
    
    layout = [[sg.Text('Your Orders:')],
              [sg.Text('Date        CartID # $  Status')],
              [sg.Listbox(orders, size = (35,20), key = 'orders')],
              [sg.Button('Back')]]

    return sg.Window('Your Orders', layout)

#-----------------------------------------------------------------------------------------
def window_paidcarts():
    global login_user_ssn
    p_carts = []
    my_carts = []
    trucks = []
    for row in cur.execute('''select plate_no
                             from Truck '''):
        trucks.append(row)
    for row in cur.execute('''select customer_id, cart_id, total_price, number_of_products, deliverer_id
                             from Cart where delivery_status = 'paid' '''):
        p_carts.append(row)
    for row in cur.execute('''select customer_id, cart_id, total_price, number_of_products
                         from Cart where delivery_status = 'paid' and deliverer_id = ? ''',(login_user_ssn,)):
        my_carts.append(row)
    layout = [[sg.Text('All Paid Carts                                                                My Carts')],
              [sg.Text('Customer Cart Total # Deliverer                                       Customer Cart Total #')],
              [sg.Listbox(p_carts,size = (30,12),key='paidcarts'),
               sg.Button('Assign to Me'),sg.Listbox(my_carts,size = (30,12),key='mycarts')],             
              [sg.Button('Go Back'),sg.Text('                                                              '),
               sg.Combo(trucks,size=(25,7),key='trucks')],
              [sg.Text('                                                                                 '),
               sg.Button('Choose Truck')]]
    
    return sg.Window('Paid Carts',layout)
    
    
#-----------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------

#buttons
def button_completeorder():
    global cart_id
    global login_user_ssn
    r = 0
    for row in cur.execute('''select Number_of_Products from Cart  
                              where cart_id = ? and customer_id = ?''',(cart_id,login_user_ssn)):
        r = row[0]
        if r!= 0:
            cur.execute('''update Cart set delivery_status = 'paid' 
                            where cart_id = ? and customer_id = ?''',(cart_id,login_user_ssn))
            con.commit()
            sg.popup('Order successfully completed!')
            passw = []
            for row in cur.execute('select password from user where ssn = ?',(login_user_ssn,)):
                passw.append(row)
            passw = passw[0][0]
            newvalue = {'ssn':login_user_ssn,'password':passw}
            button_login(newvalue)
        else:
            window = window_customer()
            sg.popup('Cart is empty!')
            
#----------------------------------------------------------------------------------------
def button_choose_truck(values):
    print(values)
    cart = values['mycarts']
    a_truck = values['trucks']
    print(cart)
    print(a_truck)
    if cart == []:
        sg.popup('Please select a cart to assign to a truck!')
    elif a_truck == '':
        sg.popup('Please chooose a truck first!')
    else:
        plate_no = a_truck[0]
        customer_id = cart[0][0]
        cart_id = cart[0][1]
        tr_query = []
        for row in cur.execute('''select * from truck_used where cart_id = ? 
                                    and customer_id=? ''',(cart_id,customer_id)):
            tr_query.append(row)
        if tr_query == []:
            cur.execute('''insert into truck_used values(?,?,?)''',(plate_no,cart_id,customer_id))
            sg.popup('Truck succesfully assigned!')
            cur.execute('''update cart set delivery_status = 'on delivery' where 
                        cart_id =? and customer_id = ?''',(cart_id,customer_id))
            con.commit()
        else: 
            sg.popup('Truck already assigned to this cart!')
        my_carts = []
        paid_carts = []
        for row in cur.execute('''select customer_id,cart_id,total_price,number_of_products
                                  from cart where
                                  deliverer_id = ?  
                                  and delivery_status = 'paid' ''',(login_user_ssn,)):
            my_carts.append(row)
        for row in cur.execute('''select customer_id, cart_id, total_price, number_of_products, deliverer_id
                             from Cart where delivery_status = 'paid' '''):
            paid_carts.append(row)
        window.Element('mycarts').Update(values=my_carts)
        window.Element('paidcarts').Update(values=paid_carts)
        con.commit()
    
#----------------------------------------------------------------------------------------    
def button_assignme(values):
    global login_user_ssn 
    to_assign = values['paidcarts']
    
    if to_assign == []:
        sg.popup('Choose a cart to assign first!') 
    else:
        customer_id = to_assign[0][0]
        cart_id = to_assign[0][1]
        paid_carts = [] 
        for row in cur.execute('''select * from cart where
                                  customer_id = ? and cart_id =? 
                                  and delivery_status = 'paid' ''',(customer_id,cart_id)):
            paid_carts.append(row)
        if paid_carts[0][1]==None:
            cur.execute('''update cart set deliverer_id = ? where 
                            customer_id = ? and cart_id =?''',(login_user_ssn,customer_id,cart_id))
            con.commit()
        else:
            sg.popup('Cart already assigned!')
            
        my_carts = []
        paid_carts = []
        for row in cur.execute('''select customer_id,cart_id,total_price,number_of_products
                                  from cart where
                                  deliverer_id = ?  
                                  and delivery_status = 'paid' ''',(login_user_ssn,)):
            my_carts.append(row)
        for row in cur.execute('''select customer_id, cart_id, total_price, number_of_products, deliverer_id
                             from Cart where delivery_status = 'paid' '''):
            paid_carts.append(row)
        window.Element('mycarts').Update(values=my_carts)
        window.Element('paidcarts').Update(values=paid_carts)
               
#-----------------------------------------------------------------------------------------

    


#-----------------------------------------------------------------------------------------





def button_addtocart(values):
    global login_user_ssn
    global cart_id 
    cart_product = values['product']
    n_amount = values['amount']
    print(cart_product)
    print(n_amount)
    
    if cart_product == []:
        sg.popup('Choose a product first!')
    elif n_amount == '':     
        sg.popup('Choose an amount first!')
    else:
        cartpno = cart_product[0][0]
        query = []
        # now let's insert this to cart. so we have to fill two tables: product_in_cart and cart
        for row in cur.execute('''select *
                                    from Product_in_cart
                                    where pnumber = ? 
                                    and cart_id = ? and customer_id = ? ''',(cartpno,cart_id,login_user_ssn)):
            query.append(row)
        if query == []:
            cur.execute('INSERT INTO Product_In_Cart Values(?,?,?,?)', (cartpno, cart_id, login_user_ssn,n_amount))
            con.commit()
        else:
            cur.execute('''update Product_in_cart set amount = amount + ? 
                        where cart_id =? and customer_id=? and pnumber = ?''',
                        (n_amount,cart_id,login_user_ssn,cartpno))
            con.commit()
        
        #initial_price = []
        n_prod_cart = 0
        add_list = []
        for row in cur.execute('''Select I.Amount, P.Price, C.total_price
                         from Product_In_Cart as I,Product as P, cart C
                         where I.cart_id = C.cart_id and 
                         I.Cart_ID = ? and I.Customer_ID = ? and P.Pnumber = I.Pnumber
                         and p.pnumber = ?''',(cart_id,login_user_ssn,cartpno)):
            #initial_price.append(row[2])
            add_list.append(row[1]*row[0])
            n_prod_cart += row[0]
        #print(initial_price)
        print(add_list)
        #total_price = initial_price + add_list
        total_price = sum(add_list)
        print(total_price)
        cur.execute('''UPDATE Cart 
                       Set Total_price = total_price + ?, Number_of_Products = Number_of_Products + ? 
                       WHERE Customer_ID = ? and Cart_ID = ?  ''', (total_price,n_prod_cart,login_user_ssn,cart_id))
        con.commit()
        sg.popup('Successfully added to Cart: ' + cart_product[0][3])
            
        
        
#-------------------------------------------------------------------------
def button_remove(values):
    global cart_id
    global login_user_ssn
    prod = values['cart']
    if prod == []:
        sg.popup('Please select a product to remove!')
    else:
        pno = prod[0][0]
        t_price = []
        n_product =[]
        newcart = []
        del_nproduct = 0
        del_tprice = 0
        for row in cur.execute('''select pi.amount,p.price from Product_In_cart pi, product p 
                                  where p.pnumber = pi.pnumber and p.Pnumber = ?
                                  and pi.customer_id = ? and pi.cart_id =? ''',(pno,login_user_ssn,cart_id)):
            del_nproduct += row[0]
            del_tprice += row[0]*row[1]
        cur.execute('''update Cart set Number_of_products = Number_of_products - ?,
                        Total_price = Total_price - ? 
                        where customer_id = ? and cart_id=?''',(del_nproduct,del_tprice,login_user_ssn,cart_id)) 
        con.commit()
        cur.execute('''delete from Product_In_cart where Pnumber = ?
                        and customer_id = ? and cart_id =? ''',(pno,login_user_ssn,cart_id))
        con.commit()
        
        for row in cur.execute('''SELECT P.Pnumber,P.Bname, P.Pname, P.price, I.amount, C.Total_price, C.Number_of_Products
                                      FROM Cart C, Product_In_Cart I, Product P
                                      WHERE C.cart_id = I.cart_id and C.customer_id = I.customer_id 
                                      and P.pnumber =I.pnumber and C.cart_id = ? and c.customer_id = ?
                                      AND P.pnumber <> ?''' ,(cart_id, login_user_ssn,pno)):
            newcart.append((row[0],row[1],row[2],row[3],row[4]))
            t_price = row[5]
            n_product = row[6]
            
        window.Element('cart').Update(values=newcart)
        window.Element('tprice').Update(value=t_price)
        window.Element('nproduct').Update(value=n_product)
        
        con.commit()


#-------------------------------------------------------------------------
def window_onetimejobs():
    global login_user_ssn
    one_time_jobs = []
    
    for row in cur.execute("SELECT W_name, Description, Date FROM Warehouse_Job WHERE Stocker_ID = ?", (login_user_ssn,)):
        one_time_jobs.append(row)
        
    layout = [[sg.Text('Your One-Time Jobs:')],
                [sg.Listbox(one_time_jobs, size=(45, len(one_time_jobs)))],
                [sg.Button('Return To Main')]]
                            
    return sg.Window('Jobs Window', layout) 
#---------------------------------------------------------------------------------------- 
def window_products():    
    brands = []    
    for row in cur.execute('''SELECT Bname
                              FROM Brand'''):
        brands.append(row)
    
    layout = [[sg.Text('All The Brands:'), sg.Combo(brands, size=(25,7), key='Brands'), sg.Button('List the Products')],
              [sg.Listbox((), size=(40,10), key='products')],
              [sg.Text('Stock Amount '), sg.Input(key='stock_amount'), sg.Button('Update Stock Amount')],
              [sg.Text('Price: '), sg.Input(key='price'), sg.Button('Update Price')],
              [sg.Button('Return To Main')]]

    return sg.Window('Product Window', layout)

#-------------------------------------------------------------------------------------
def button_list_theproducts(values):
    
    brand = values["Brands"]
    if brand == '':
        sg.popup('Please choose a brand.')
    else:
        brand_name = brand[0]
                            
        pproducts = []
        
        for row in cur.execute('''SELECT Product.Pnumber, Product.Description, Product.Pname, Product.price, Product.Stock_amt, Product.Unit
                                  FROM Product, Brand
                                  WHERE ? = Product.Bname AND Product.Bname = Brand.Bname 
                                  ''', (brand_name,)):
            pproducts.append(row)
        print(pproducts)
        window.Element('products').Update(values=pproducts)
#---------------------------------------------------------------------------------------
def button_update_price(values):
    brand = values['Brands']
    product = values['products']
    price = values['price']
    if brand == '':
        sg.popup('Please choose a brand first.')
    elif product == []:
        sg.popup('Please choose a product first.')
    elif price == '':
        sg.popup('Please enter price.')
    elif not price.isnumeric():
        sg.popup('Price should be numeric!')
    else:
        price_value = int(price)
        if price_value < 0:
             sg.popup('Price cannot be negative!')
        else:
            product_number = product[0][0]
            brand_name = brand[0]
            
            cur.execute('UPDATE Product SET price = ? WHERE Pnumber = ? AND Bname = ?', (price_value, product_number, brand_name))
            con.commit()
            sg.popup('Successfully set price to ' + price)
            
            
            window.Element('price').Update(value='')
            
            
            products = []
            for row in cur.execute('''SELECT Product.Pnumber, Product.Description, Pname, price, Stock_amt, Unit
                                      FROM Brand, Product
                                      WHERE ? = Brand.Bname AND Brand.Bname = Product.Bname
                                      ''', (brand_name,)):
                products.append(row)
            window.Element('products').Update(values=products)     
#--------------------------------------------------------------------------------------
def button_update_stock(values):
    brand = values['Brands']
    product = values['products']
    stock = values['stock_amount']
    if brand == '':
        sg.popup('Please choose a brand first.')
    elif product == []:
        sg.popup('Please choose a product first.')
    elif stock == '':
        sg.popup('Please enter stock amount.')
    elif not stock.isnumeric():
        sg.popup('Stock amount should be numeric!')
    else:
        stock_value = int(stock)
        if stock_value < 0:
             sg.popup('Stock amount cannot be negative!')
        else:
            product_number = product[0][0]
            brand_name = brand[0]
            
            cur.execute('UPDATE Product SET Stock_amt = ? WHERE Pnumber = ? AND Bname = ?', (stock_value, product_number, brand_name))
            con.commit()
            sg.popup('Successfully set stock amount to ' + stock)
            
       
            window.Element('stock_amount').Update(value='')
            
          
            products = []
            for row in cur.execute('''SELECT Product.Pnumber, Product.Description, Pname, price, Stock_amt, Unit
                                      FROM Brand, Product
                                      WHERE ? = Brand.Bname AND Brand.Bname = Product.Bname
                                      ''', (brand_name,)):
                products.append(row)
            print(products)
            window.Element('products').Update(values=products)  

#-----------------------------------------------------------------------------------------
def window_new_product():                            
    brands = []
    
    for row in cur.execute('''SELECT Bname
                              FROM Brand'''):
        brands.append(row)
    
    layout = [[sg.Text('Choose Brand:',size=(10,1)), sg.Combo(brands, size=(25,7), key='Brand')],
              [sg.Text('Product Name:',size=(10,1)), sg.Input(key='pname',size=(10,1))],
              [sg.Text('Product Description:',size=(10,1)), sg.Input(key='p_description',size=(10,1))],
              [sg.Text('Unit Price:',size=(10,1)), sg.Input(key='unit_price',size=(10,1))],
              [sg.Text('Stock Amount:',size=(10,1)), sg.Input(key='stock_amt',size=(10,1))],
              [sg.Text('Unit Description:',size=(10,1)), sg.Input(key='u_description',size=(10,1))],
              [sg.Button('Insert'), sg.Button('Return To Main')]]

    return sg.Window('Insert Window', layout)
#-------------------------------------------------------------------------------------------
def button_insert(values):
    
    brand = values["Brand"]
    name = values['pname']
    pdescription = values['p_description']
    uprice = values['unit_price']
    stock_amt = values['stock_amt']
    udescription = values['u_description']                        
    if brand == '':
        sg.popup('Brand cannot be empty!')
    elif name == '':
        sg.popup('Product name cannot be empty!')
    elif pdescription == '':
        sg.popup('Please write a description or put 0 for no description')
    elif udescription == '':
        sg.popup('Please write a description or put 0 for no description')   
    elif uprice == '':
        sg.popup('Please write a price or write 0 for no specified price')
    elif stock_amt == '':
        sg.popup('Please write a stock amount or write 0 for no specified stock amount')                        
    elif not uprice.isnumeric():
        sg.popup('Unit price should be numeric.')
    elif not stock_amt.isnumeric():
        sg.popup('Stock amount should be numeric.')                         
    else:
        stock_amt_value = int(stock_amt)
        uprice_value = int(uprice)                    
        if uprice_value < 0:
            sg.popup('Unit price cannot be negative')
        elif stock_amt_value < 0:
            sg.popup('Stock amount cannot be negative')  
        else:
            cur.execute('SELECT MAX(Pnumber) FROM Product')
            row = cur.fetchone()

            if row is None:
                new_number = 1
            else:
                new_number = row[0] + 1
    
            cur.execute('INSERT INTO Product VALUES (?,?,?,?,?,?,?)',(new_number,pdescription,name,uprice_value,stock_amt_value,udescription,)+ brand)
            con.commit()
            sg.popup('Successfully inserted ' + name + ' ' + pdescription + ' with number: ' + str(new_number) + " " + 'and price/stock amount values ' + str(uprice_value) + " " + str(stock_amt_value))
            window.Element('Brand').Update(value='')
            window.Element('pname').Update(value='')
            window.Element('p_description').Update(value='')
            window.Element('unit_price').Update(value='')
            window.Element('stock_amt').Update(value='')
            window.Element('u_description').Update(value='') 
#--------------------------------------------------------------------------------------------
def button_login(values):
    
    global login_user_ssn
    global login_user_name
    global login_user_type
    global window
    global cart_id
    
    ussn = values['ssn']
    upass = values["password"]
    if ussn == '':
        sg.popup('SSN cannot be empty')
    elif upass == '':
        sg.popup('Password cannot be empty')
    else:
        # first check if this is a valid user
        cur.execute('SELECT SSN, Name FROM User WHERE SSN = ? AND Password = ?', (ussn,upass))
        row = cur.fetchone()
        
        if row is None:
            sg.popup('SSN or Password is wrong!')
        else:
            
            login_user_ssn = row[0]
            login_user_name = row[1]
            # let's first check if this is a customer
            cur.execute('SELECT SSN FROM Customer WHERE SSN = ?', (ussn,))
            row_customer = cur.fetchone()
            
            if row_customer is None:
                # this is not a customer, let's check for deliverer
                cur.execute('SELECT SSN FROM Deliverer WHERE SSN = ?', (ussn,))
                row_deliverer = cur.fetchone()
                if row_deliverer is None:
                    # this is not a deliverer also, then check for stocker
                    cur.execute('SELECT SSN FROM Stocker WHERE SSN = ?', (ussn,))
                    row_stocker = cur.fetchone()
                    
                    if row_stocker is None:
                        sg.popup('Something went wrong! Try Again!')
                    else:
                        #is a stocker
                        login_user_type = 'Stocker'
                        sg.popup('Welcome, ' + login_user_name + ' (Stocker)')
                        window.close()
                        window = window_stocker()
                    
                else:
                    #is a deliverer
                    login_user_type = 'Deliverer'
                    sg.popup('Welcome, ' + login_user_name + ' (Deliverer)')
                    window.close()
                    window = window_deliverer()
            else:
                # this is a customer
                    login_user_type = 'Customer'
                    sg.popup('Welcome, ' + login_user_name + ' (Customer)')
                    window.close()
                    cart_id = random.randint(101,999)
                    window = window_customer()
                    date = datetime.today().strftime('%Y-%m-%d')
                    cur.execute('INSERT INTO Cart Values(?,null,?,?,0,?,0)', (login_user_ssn,cart_id,date,'new'))
                    con.commit()
                    
                    
window = window_login()
#--------------------------------------------------------------------------
#--------------------------------------------------------------------------
#--------------------------------------------------------------------------
while True:
    event, values = window.read()
    if event == 'Login':
        cart_id = 0
        button_login(values)
    elif event == 'Shop':
        window.close()
        window = window_shop()
    elif event == 'Return To Main':
        window.close()
        window = window_stocker()
    elif event == 'Back':
        window.close()
        window = window_customer()
    elif event == 'Go Back':
        window.close()
        window = window_deliverer()
    elif event == 'Add to Cart':
        button_addtocart(values)
    elif event == 'Cart':
        window.close()
        window = window_cart()
    elif event == 'Paid Carts':
        window.close()
        window = window_paidcarts()
    elif event == 'Assign to Me':
        button_assignme(values)
    elif event == 'Remove a Product':
        print(values)
        button_remove(values)
    elif event == 'Main Page':
        window.close()
        window = window_deliverer()
    elif event == 'Orders':
        window.close()
        window = window_orders()
    elif event == 'Logout':
        # set login user global parameters
        login_user_id = -1
        login_user_name = -1
        login_user_type = -1
        cur.execute('''delete from cart where number_of_products = 0 
                       or total_price = 0''')
        con.commit()
        window.close()
        window = window_login()
    elif event == 'List Products':
        button_list_products(values)
    elif event == 'Choose Truck':
        button_choose_truck(values)
    elif event == 'Complete Order':
        window.close()
        button_completeorder()
    elif event == 'Update Product Information':
        window.close()
        window = window_products()
    elif event == 'Ongoing Deliveries':
        window.close()
        window = window_deliveries()
    elif event == 'Completed Deliveries':
        window.close()
        window = window_completed_deliveries()
    elif event == 'Your One-Time Jobs':
        window.close()
        window = window_onetimejobs() 
    elif event == 'Show Cart':
        button_show_cart(values)
    elif event == 'Mark as Completed':
        button_markascompleted(values)
    elif event == 'Update Price':
        button_update_price(values)
    elif event == 'Update Stock Amount':
        button_update_stock(values)
    elif event == 'List the Products':
        button_list_theproducts(values)
    elif event == 'Insert New Product':
        window.close()
        window = window_new_product()                    
    elif event == 'Insert':
        button_insert(values)
    elif event == sg.WIN_CLOSED:
        cur.execute('''delete from cart where number_of_products = 0 
                       or total_price = 0''')
        con.commit()
        break

window.close()

con.commit()
con.close()

{'paidcarts': [], 'mycarts': [], 'trucks': ('034587',)}
[]
('034587',)
{'paidcarts': [], 'mycarts': [(134234098, 506, 56.0, 2)], 'trucks': ('034587',)}
[(134234098, 506, 56.0, 2)]
('034587',)
{'dcarts': (134234098, 506), 'addcarts': []}
